In [451]:
pre_folder = r'M:\ISI result\HGA\LTS-03129\PRE\DATA.Production.1.csv'
post_folder = r'M:\ISI result\HGA\LTS-03129\POST\DATA.Production.1.csv'
out_path = r'M:\ISI result\HGA\LTS-03129\ISI_HAST2.xlsx'

In [452]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference

In [453]:
pre=pd.read_csv(pre_folder)
pre['Hd&Part ID'] = pre['Hd']+pre['Part ID']
#grouppid = pre.groupby('Hd&Part ID')
#pre=grouppid[['Hd','Part ID','SBR','RESISTANCE','AMPLITUDE','ASYMETRY','BARK_JMP','BARK_JMP_UV','HYSTERESIS','SLOPE','SMANMAX_MAX','SMANAMP_AVG','SMANRMS_AVG','SMAN_NORM']].tail(1)

In [454]:
prescreen = pre.loc[(pre['RESISTANCE'] < 100)|(pre['RESISTANCE'] > 1000)]
pre = pre.loc[(pre['RESISTANCE'] > 100)&(pre['RESISTANCE'] < 1000)]

In [455]:
post=pd.read_csv(post_folder)
post['Hd&Part ID'] = post['Hd']+post['Part ID']
grouppid = post.groupby('Hd&Part ID')
post=grouppid[['Hd','Part ID','RESISTANCE','AMPLITUDE','ASYMETRY','BARK_JMP','BARK_JMP_UV','HYSTERESIS','SLOPE','SMANMAX_MAX','SMANAMP_AVG','SMANRMS_AVG','SMAN_NORM']].tail(1)

In [456]:
pre_post = pd.merge(pre,post,how='left', on=['Hd','Part ID'] , suffixes=('_PRE','_POST'))
pre_post = pre_post .drop(['Hd&Part ID'], axis=1)

In [457]:
pre_post['%dMRE'] = (pre_post['RESISTANCE_POST']/pre_post['RESISTANCE_PRE'] -1)*100
pre_post['%dAMP'] = (pre_post['AMPLITUDE_POST']/pre_post['AMPLITUDE_PRE'] -1)*100
pre_post['%dASYM'] = pre_post['ASYMETRY_POST']-pre_post['ASYMETRY_PRE']
pre_post['%dBJ'] = pre_post['BARK_JMP_POST']-pre_post['BARK_JMP_PRE']
pre_post['%dSMM'] = (pre_post['SMANMAX_MAX_POST']/pre_post['SMANMAX_MAX_PRE'] -1)*100

In [458]:
missingid = pre_post[pre_post['%dMRE'].isnull()]
missing = pre_post.dropna(subset=['%dMRE'])
pre_post = missing

In [459]:
pre_post.loc[abs(pre_post['%dMRE'])>10,'DISPOSITION'] = "RES"
pre_post.loc[(abs(pre_post['%dMRE'])<10)&(abs(pre_post['%dAMP'])>20),'DISPOSITION'] = "AMP"
pre_post.loc[(abs(pre_post['%dMRE'])<10)&(abs(pre_post['%dAMP'])<20)&(abs(pre_post['%dASYM'])>10),'DISPOSITION'] = "ASYM"
pre_post.loc[(abs(pre_post['%dMRE'])<10)&(abs(pre_post['%dAMP'])<20)&(abs(pre_post['%dASYM'])<10)&(abs(pre_post['%dBJ'])>50)&(abs(pre_post['BARK_JMP_POST'])>15),'DISPOSITION'] = "BARK_JMP"
pre_post.loc[(abs(pre_post['%dMRE'])<10)&(abs(pre_post['%dAMP'])<20)&(abs(pre_post['%dASYM'])<10)&(abs(pre_post['%dBJ'])<50)&(abs(pre_post['BARK_JMP_POST'])<15)&(abs(pre_post['%dSMM'])>50)&(abs(pre_post['SMAN_NORM_POST'])>30),'DISPOSITION'] = "SMAN"
pre_post.loc[(abs(pre_post['%dMRE'])<10)&(abs(pre_post['%dAMP'])<20)&(abs(pre_post['%dASYM'])<10)&(abs(pre_post['%dBJ'])<50)&(abs(pre_post['BARK_JMP_POST'])<15)&(abs(pre_post['%dSMM'])<50)&(abs(pre_post['SMAN_NORM_POST'])<30)&(pre_post['SLOPE_PRE']>0)&(pre_post['SLOPE_POST']>0),'DISPOSITION'] = "SLOPE"

In [460]:
pre_post["DISPOSITION"].fillna("NORMAL", inplace = True)

In [461]:
table = pd.pivot_table(pre_post, values='Part ID', index=['SBR'], columns=['DISPOSITION'], aggfunc='count', margins=False)

In [462]:
table['SAMPLE'] = table.sum(axis = 1)
table['%FAIL_RATE'] = (table['SAMPLE']-table['NORMAL'])/table['SAMPLE']*100

In [463]:
PRE_SUM = pre.groupby(['SBR','Hd']).agg({'RESISTANCE': ['count','mean','std'],'AMPLITUDE': ['mean','std'],'ASYMETRY': ['mean','std'],'BARK_JMP': ['mean','std'],'HYSTERESIS': ['mean','std'],'SLOPE': ['mean','std'],'SMANMAX_MAX': ['mean','std'],'SMANAMP_AVG': ['mean','std'],'SMANRMS_AVG': ['mean','std'],'SMAN_NORM': ['mean','std']})


In [464]:
PRE_SUM = pd.DataFrame(PRE_SUM)
PRE_SUM = PRE_SUM.T

In [465]:
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')

In [466]:
pre_post.to_excel(writer, sheet_name='ALL', index=None)
prescreen.to_excel(writer, sheet_name='PRE_SCREENING')
PRE_SUM.to_excel(writer, sheet_name='PRELIM')
missingid.to_excel(writer, sheet_name='MISSING_DATA')
table.to_excel(writer, sheet_name='FAIL_RATE')

In [467]:
writer.save()